# Creating bins: Ganglion Cells in the Retina
- **Author:** David Felipe
- **Contact:** https://github.com/davidnfu0
- **Last Modification:** January 25, 2024
- **Description:** In this file, the spikes will be separated into bins, with the aim of facilitating subsequent analysis.

## Introduction

### Importing libraries

In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
from pyret import spiketools as spkt

In [2]:
sys.path.append('../../')

In [3]:
from scripts import load_yaml_config

### Paths, constants and configuration

In [4]:
configPath = "../../config/"
config = load_yaml_config(configPath + "general_config.yml")

NEW_EVENT_LIST_PATH = "../../" + config["paths"]["data"]["new_event_list"]
STIM_DICT_PATH = "../../" + config["paths"]["data_cache"]["STIM_DICT"]

### Loading data

In [5]:
NEW_EVENT_LIST = pd.read_csv(NEW_EVENT_LIST_PATH)

with open(STIM_DICT_PATH, "rb") as file:
    STIM_DICT = pickle.load(file)

## Creating bins

In [6]:
binsStimDict = {}
PROTOCOLS_DESCRIPTION = list(STIM_DICT[list(STIM_DICT.keys())[0]].keys())
for protocolDescription in PROTOCOLS_DESCRIPTION:
    protocol, description = protocolDescription.split(".")
    switcher = {
        "chirp": 40,
        "scotopic": 1,
        "natural_images": 1000,
        "ensembles": 1000,
        "checkerboard": 1000,
        "photopic": 1,
    }
    binsStimDict[protocolDescription] = switcher.get(protocol, 1000)
binsStimDict["chirp.ON"] = 60
binsStimDict["chirp.OFF"] = 60
binsStimDict["chirp.FREQ"] = 300
binsStimDict["chirp.AMP"] = 160

In [ ]:
rasterStimDict = {}
for temp, events in STIM_DICT.items():
    rasterStimDict[temp] = {}
    for protocolDescription, repetitions in events.items():
        protocol, description = protocolDescription.split(".")
        rasterStimDict[temp][protocolDescription] = {}
        for repetition, spikes in repetitions.items():
            if len(spikes) == 0:
                rasterStimDict[temp][protocolDescription][repetition] = np.array([0])
                continue
            elif binsStimDict[protocolDescription] == 1:
                rasterStimDict[temp][protocolDescription][repetition] = np.array([len(spikes)])
                continue
            filtered_events = NEW_EVENT_LIST.loc[
                                                    (NEW_EVENT_LIST["protocol_name"] == protocol) &
                                                    (NEW_EVENT_LIST["extra_description"] == description) &
                                                    (NEW_EVENT_LIST["repetition_name"] == repetition)
                                                ]
            star_time = filtered_events["start_event"]
            end_time = filtered_events["end_event"]
            time = np.linspace(0, end_time - star_time, binsStimDict[protocolDescription])
            rasterStimDict[temp][protocolDescription][repetition] = spkt.binspikes(spikes, time)

## Export data

In [9]:
with open("../../" + config["paths"]["data_cache"]["BINS_STIM_DICT"], 'wb') as output: 
    pickle.dump(rasterStimDict, output)

____